<a href="https://colab.research.google.com/github/chaudharynidhi/Tensorflow_DataPreProcessing/blob/master/chapter_13_Q9%2C_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
X_train, y_train = X_train_full[5000:], y_train_full[5000:]
X_valid, y_valid = X_train_full[:5000], y_train_full[:5000]

In [ ]:
train_shuffled = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train))
valid = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
def create_example(image, label):
  image = tf.io.serialize_tensor(image)

  return Example(features = Features(
      feature={
          "image": Feature(bytes_list = BytesList(value=[image.numpy()])),
          "label": Feature(int64_list=Int64List(value=[label])),
      }
  ))

In [ ]:
Example = tf.train.Example
Features = tf.train.Features
Feature = tf.train.Feature
BytesList = tf.train.BytesList
Int64List = tf.train.Int64List

In [ ]:
for image, label in train_shuffled.take(1):
  print(create_example(image, label))

features {
  feature {
    key: "image"
    value {
      bytes_list {
        value: "\010\004\022\010\022\002\010\034\022\002\010\034\"\220\006\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\00

In [ ]:
from contextlib import ExitStack

def write_tfrecords(dataset, name, n_shards = 10):
  paths = ["{}.tfrecord-{:05d}-of-{:05d}".format(name, index, n_shards) for index in range(n_shards)]

  with ExitStack() as stack:
    writers = [stack.enter_context(tf.io.TFRecordWriter(path)) for path in paths]
    for index, (image,label) in dataset.enumerate():
      shard = index % n_shards
      example = create_example(image, label)
      writers[shard].write(example.SerializeToString())

  return paths

In [ ]:
train_filepaths = write_tfrecords(train_shuffled, "my_train_tf_record")
valid_filepaths = write_tfrecords(valid, "my_valid_tf_record")
test_filepaths = write_tfrecords(test, "my_test_tf_record")

In [ ]:
def preprocess(tfrecord):
  feature_descriptions = {
      "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
      "label": tf.io.FixedLenFeature([], tf.int64, default_value = -1)
  }
  example = tf.io.parse_single_example(tfrecord, feature_descriptions)
  image = tf.io.parse_tensor(example["image"], out_type=tf.uint8)
  image = tf.reshape(image, shape=[28,28])
  return image, example['label']

def mnist_dataset(filepaths, n_read_threads=5, shuffle_buffer_size = None, n_parse_threads = 5, batch_size=32, cache=True):
  dataset = tf.data.TFRecordDataset(filepaths, num_parallel_reads=n_read_threads)
  if cache:
    dataset = dataset.cache()
  if shuffle_buffer_size:
    dataset = dataset.shuffle(shuffle_buffer_size)
  dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
  dataset = dataset.batch(batch_size)
  return dataset.prefetch(1)

In [ ]:
train_set = mnist_dataset(train_filepaths, shuffle_buffer_size=60000)
valid_set = mnist_dataset(valid_filepaths)
test_set = mnist_dataset(test_filepaths)

In [ ]:
class Standardization(keras.layers.Layer):
  def adapt(self,data_sample):
    self.mean_ = np.mean(data_sample, axis=0, keepdims=True)
    self.variance_ = np.std(data_sample, axis=0, keepdims=True)
  def call(self, inputs):
    return (inputs-self.mean_)/(self.variance_+keras.backend.epsilon())

In [ ]:
standardization = Standardization(input_shape=[28,28])

In [ ]:
sample_image_train = train_set.take(1000).map(lambda image, label: image)
sample_images = np.concatenate(list(sample_image_train.as_numpy_iterator()), axis=0).astype(np.float32)

standardization.adapt(sample_images)

model = keras.Sequential([
                          standardization,
                          keras.layers.Flatten(),
                          keras.layers.Dense(100, activation='relu'),
                          keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer = 'nadam', metrics = ['accuracy'])

In [ ]:
model.fit(train_set, epochs=10, validation_data=valid_set)

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5466 - accuracy: 0.8098 - val_loss: 0.3685 - val_accuracy: 0.8762
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3345 - accuracy: 0.8797 - val_loss: 0.3647 - val_accuracy: 0.8780
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2882 - accuracy: 0.8942 - val_loss: 0.3622 - val_accuracy: 0.8766
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2657 - accuracy: 0.9043 - val_loss: 0.3554 - val_accuracy: 0.8792
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2358 - accuracy: 0.9141 - val_loss: 0.3619 - val_accuracy: 0.8794
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2275 - accuracy: 0.9159 - val_loss: 0.3699 - val_accuracy: 0.8778
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2089 - accuracy: 0.9215 - val_loss: 0.3763 - val_accuracy: